In [1]:
!git clone https://github.com/microsoft/SoftTeacher.git

Cloning into 'SoftTeacher'...
remote: Enumerating objects: 296, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 296 (delta 73), reused 60 (delta 58), pack-reused 202
Receiving objects: 100% (296/296), 590.38 KiB | 18.45 MiB/s, done.
Resolving deltas: 100% (140/140), done.


In [2]:
!pip3 install mmcv==1.3.9

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 313 kB 28.9 MB/s 
     |████████████████████████████████| 190 kB 64.3 MB/s 
  Created wheel for mmcv: filename=mmcv-1.3.9-py2.py3-none-any.whl size=451832 sha256=c9eafa953d75fa523091be81fa9e3504b2293be556285e7e0d3e5f6e285f74ed
  Stored in directory: /root/.cache/pip/wheels/88/48/bf/655e136aea5534d7a9a85fe247fee7957178fc19cf79dda602
Successfully built mmcv


In [3]:
!ls

sample_data  SoftTeacher


In [4]:
cd SoftTeacher

/content/SoftTeacher


In [ ]:
!make install

make pre
make[1]: Entering directory '/content/SoftTeacher'
python -m pip install -r requirements.txt -f https://download.pytorch.org/whl/torch_stable.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Using cached mmcv-full-1.6.1.tar.gz (563 kB)
  Using cached wandb-0.13.1-py2.py3-none-any.whl (1.8 MB)
  Using cached pathtools-0.1.2.tar.gz (11 kB)
  Using cached docker_pycreds-0.4.0-py2.py3-none-any.whl (9.0 kB)
  Using cached setproctitle-1.3.1-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (30 kB)
  Using cached shortuuid-1.0.9-py3-none-any.whl (9.4 kB)
  Using cached sentry_sdk-1.9.2-py2.py3-none-any.whl (157 kB)
  Using cached GitPython-3.1.27-py3-none-any.whl (181 kB)
  Using cached gitdb-4.0.9-py3-none-any.whl (63 kB)
  Using cached smmap-5.0.0-py3-none-any.whl (24 kB)
  Using cached sentry_sdk-1.9.1-py2.py3

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!python demo/image_demo.py /content/014.png configs/soft_teacher/soft_teacher_faster_rcnn_r50_caffe_fpn_coco_full_720k.py /content/drive/MyDrive/Colab_Notebooks/pytorch/code/lab/plenoptic/pretrained/iter_720000.pth --output /content/

/content/SoftTeacher/thirdparty/mmdetection/mmdet/datasets/pipelines/formating.py:7: UserWarning: DeprecationWarning: mmdet.datasets.pipelines.formating will be deprecated, please replace it with mmdet.datasets.pipelines.formatting.
  warnings.warn('DeprecationWarning: mmdet.datasets.pipelines.formating will be '
load checkpoint from local path: /content/drive/MyDrive/Colab_Notebooks/pytorch/code/lab/plenoptic/pretrained/iter_720000.pth
/content/SoftTeacher/thirdparty/mmdetection/mmdet/datasets/utils.py:70: UserWarning: "ImageToTensor" pipeline is replaced by "DefaultFormatBundle" for batch inference. It is recommended to manually replace it in the test data pipeline in your config file.
  'data pipeline in your config file.', UserWarning)
Save results to /content/014.png


###demo.py

In [ ]:
import asyncio
import glob
import os
from argparse import ArgumentParser

from mmcv import Config
from mmdet.apis import async_inference_detector, inference_detector, show_result_pyplot

from ssod.apis.inference import init_detector, save_result
from ssod.utils import patch_config


def parse_args():
    parser = ArgumentParser()
    parser.add_argument("img", help="Image file")
    parser.add_argument("config", help="Config file")
    parser.add_argument("checkpoint", help="Checkpoint file")
    parser.add_argument("--device", default="cuda:0", help="Device used for inference")
    parser.add_argument(
        "--score-thr", type=float, default=0.3, help="bbox score threshold"
    )
    parser.add_argument(
        "--async-test",
        action="store_true",
        help="whether to set async options for async inference.",
    )
    parser.add_argument(
        "--output",
        type=str,
        default=None,
        help="specify the directory to save visualization results.",
    )
    args = parser.parse_args()
    return args


def main(args):
    cfg = Config.fromfile(args.config)
    # Not affect anything, just avoid index error
    cfg.work_dir = "./work_dirs"
    cfg = patch_config(cfg)
    # build the model from a config file and a checkpoint file
    model = init_detector(cfg, args.checkpoint, device=args.device)
    imgs = glob.glob(args.img)
    for img in imgs:
        # test a single image
        result = inference_detector(model, img) # 이 부분을 저장
        # show the results
        if args.output is None:
            show_result_pyplot(model, img, result, score_thr=args.score_thr)
        else:
            out_file_path = os.path.join(args.output, os.path.basename(img))
            print(f"Save results to {out_file_path}")
            save_result(
                model, img, result, score_thr=args.score_thr, out_file=out_file_path
            )


async def async_main(args):
    cfg = Config.fromfile(args.config)
    # Not affect anything, just avoid index error
    cfg.work_dir = "./work_dirs"
    cfg = patch_config(cfg)
    # build the model from a config file and a checkpoint file
    model = init_detector(cfg, args.checkpoint, device=args.device)
    # test a single image
    args.img = glob.glob(args.img)
    tasks = asyncio.create_task(async_inference_detector(model, args.img))
    result = await asyncio.gather(tasks)
    # show the results
    for img, pred in zip(args.img, result):
        if args.output is None:
            show_result_pyplot(model, img, pred, score_thr=args.score_thr)
        else:
            out_file_path = os.path.join(args.output, os.path.basename(img))
            print(f"Save results to {out_file_path}")
            save_result(
                model, img, pred, score_thr=args.score_thr, out_file=out_file_path
            )


if __name__ == "__main__":
    args = parse_args()
    if args.async_test:
        asyncio.run(async_main(args))
    else:
        main(args)